In [15]:
import pandas as pd 
import numpy as np 
from pprint import pprint
import matplotlib.pyplot as plt
import sys,os
import itertools

In [7]:
direc = 'All_Countries_Representative_Seq'
Other_Info_File = "All_Train_Test_Gisaid.csv"

ACC_VECTS_direc = 'All_Countries_Spectral_Vects'
Other_Info_df = pd.read_csv(Other_Info_File)
Other_Info_df_2 = pd.read_csv("gisaid_new_data.csv")
# ref_seq_df = pd.read_csv("All_Countries_Splitted/Ref_seq.csv")
df_list = []
# cols = ['Country','Accession ID', 'Fast Vector', 'Virus name', 'Location',
#        'Collection date', 'Death', 'Indicator', 'Sequence']
cols = ['Country','Accession ID', 'Spectral Vector', 'Virus name', 'Location',
       'Collection date', 'Sequence']
for file_ in os.listdir(direc):
    typ = file_.split(".")[1]
    if(typ == 'txt'):
        cont_ = file_.split("_")[0]
        print("working with->",cont_)
        Vect_file = ACC_VECTS_direc +"/"+cont_+"_Euclidean.csv"
        c_fl = open(direc+"/"+file_,"r")
        center_id = c_fl.read()
        print("center",center_id)
        Fast_vector_df = pd.read_csv(Vect_file)

        final_df_1 = Fast_vector_df.loc[Fast_vector_df["Accession ID"] == center_id]
        final_df_2 = Other_Info_df.loc[Other_Info_df["Accession ID"] == center_id]
        # print()
        if(final_df_2.shape[0] == 0):
            final_df_2 = Other_Info_df_2[Other_Info_df_2["Accession ID"] == center_id]
        # if(final_df_2.shape[0] == 0):
        #     final_df_2 = ref_seq_df[ref_seq_df["Accession ID"] == center_id]
        if(cont_ == "United Kingdom" or cont_ == "Korea"):
            continue
        # frame = {'Accession ID':pd.Series(center_id),'Country':pd.Series(cont_)}
        # final_df_3 = pd.DataFrame(frame)
        # print(final_df_1.shape)
        # print(final_df_2.shape)
        final_df = pd.merge(final_df_1, final_df_2, on='Accession ID')
        final_df["Country"] = cont_
        final_df = final_df[cols]
        df_list.append(final_df)

f_df = pd.concat(df_list,ignore_index=True)
print(f_df.shape)

working with-> Algeria
center EPI_ISL_420037
working with-> Argentina
center EPI_ISL_420600
working with-> Australia
center EPI_ISL_426990
working with-> Austria
center EPI_ISL_419662
working with-> Bangladesh
center EPI_ISL_445214
working with-> Belarus
center EPI_ISL_419693
working with-> Belgium
center EPI_ISL_421188
working with-> Brazil
center EPI_ISL_427305
working with-> Cambodia
center EPI_ISL_411902
working with-> Canada
center EPI_ISL_418344
working with-> Chile
center EPI_ISL_414579
working with-> China
center EPI_ISL_416473
working with-> Colombia
center EPI_ISL_418262
working with-> Czech Republic
center EPI_ISL_426887
working with-> Democratic Republic of the Congo
center EPI_ISL_417441
working with-> Ecuador
center EPI_ISL_422565
working with-> England
center EPI_ISL_425337
working with-> Estonia
center EPI_ISL_420066
working with-> Finland
center EPI_ISL_418389
working with-> France
center EPI_ISL_418429
working with-> Georgia
center EPI_ISL_415644
working with-> Ghana


In [8]:
print(f_df.columns)
f_df.to_csv("All_country_representative_seq_info.csv",index=False)

Index(['Country', 'Accession ID', 'Spectral Vector', 'Virus name', 'Location',
       'Collection date', 'Sequence'],
      dtype='object')


In [21]:
def get_motifs(length,sequences_list):
    (d,index) = ({}, 0)
    for seq in sequences_list:
        for i in range(0, len(seq)-length+1):
            word = seq[i:i+length]
            if word not in d:
                d[word] = index
                index += 1 
    return d

def calculate_occurrences(length, df):
    # df = pd.read_csv(input_file)
    sequences_list = df["Sequence"]  #get_sequences(input_file)
    d = get_motifs(length,sequences_list)
    print("Done with motif finding")   
    rows_num = len(sequences_list)
    cols_num = len(d)
    data = np.zeros(shape=(rows_num,cols_num))
    for row_idx, seq in enumerate(sequences_list):
        for i in range(0, len(seq)-length+1):
            word = seq[i:i+length]
            col_idx = d[word]
            data[row_idx, col_idx] += 1
    return data

In [13]:
def angle_cos(i,j,list_):
    return np.sum(list_[i,:] * list_[j,:])/(np.sqrt(np.sum(list_[i,:]**2)) * np.sqrt(np.sum(list_[j,:]**2)))

def cosine(list_, seqs_number):
    matrix = np.zeros([seqs_number, seqs_number])
    for i, j in itertools.combinations(range(0,seqs_number),2):
         matrix[i][j] = matrix[j][i] = (1 - angle_cos(i,j,list_)).round(10) + 0.0
    return matrix

In [23]:
f_df = pd.read_csv("All_country_representative_seq_info.csv")
def all_country_distance_table():
    vect = f_df['Spectral Vector']
    cont_ = f_df['Country']
    final_vect_len = f_df.shape[0]
    print(final_vect_len)
    # final_vect = np.full((final_vect_len,final_vect_len),-89)
    list_ = []
    # for i in range(final_vect_len):
    #     var_1 = np.asarray(vect[i].replace('"',"").replace("[","").replace("]","").replace(' ','').split(",")).astype(np.float)
    #     list_.append(var_1)
    list_ = calculate_occurrences(final_vect_len,f_df)
    print(list_)
    final_vect = cosine(list_,final_vect_len)
        # for j in range(final_vect_len):
        #     # print("Doing -> ",cont_[i],cont_[j])
        #     var_1 = 
        #     var_2 = np.asarray(vect[j].replace('"',"").replace("[","").replace("]","").replace(' ','').split(",")).astype(np.float)
        #     final_vect[i][j] = (1-(np.sum(var_1 * var_2)/(np.sqrt(np.sum(var_1**2)) * np.sqrt(var_2**2)))).round(10) + 0.0
            # print(final_vect[i][j])

    # print(final_vect)
    return (np.array(cont_),final_vect)

(cont_,final_vect) = all_country_distance_table()
print(cont_)
final_col = []
final_col.append("")
final_col.extend(cont_)
cont_ = cont_.reshape(cont_.shape[0],1)
target = np.hstack((cont_, final_vect))
print(target.shape)
ekdom_final_df = pd.DataFrame(target,columns=final_col)
# print(ekdom_final_df.shape)
# ekdom_final_df.to_csv("Final_distance_matrix.csv",index=False)

68
Done with motif finding
[[1. 1. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 1. 1.]]
['Algeria' 'Argentina' 'Australia' 'Austria' 'Bangladesh' 'Belarus'
 'Belgium' 'Brazil' 'Cambodia' 'Canada' 'Chile' 'China' 'Colombia'
 'Czech Republic' 'Democratic Republic of the Congo' 'Ecuador' 'England'
 'Estonia' 'Finland' 'France' 'Georgia' 'Ghana' 'Greece' 'Hong Kong'
 'Hungary' 'Iceland' 'India' 'Iran' 'Ireland' 'Israel' 'Italy' 'Japan'
 'Kuwait' 'Latvia' 'Lithuania' 'Luxembourg' 'Malaysia' 'Mexico' 'Nepal'
 'Netherlands' 'New Zealand' 'Nigeria' 'Norway' 'Pakistan' 'Denmark'
 'Germany' 'Panama' 'Singapore' 'Switzerland' 'Peru' 'Poland' 'Portugal'
 'Qatar' 'Russia' 'Saudi Arabia' 'Senegal' 'Slovakia' 'Slovenia'
 'South Africa' 'South Korea' 'Spain' 'Sweden' 'Taiwan' 'Thailand'
 'Turkey' 'Uruguay' 'USA' 'Vietnam']
(68, 69)
